In [1]:
import pytrends
from pytrends.request import TrendReq

pytrend = TrendReq()

def get_google_trends_score(title):
    # Get the Google Trends data for the given title
    try:
        pytrend.build_payload([title])
        trend_data = pytrend.interest_over_time()
        
        # Calculate the point value based on the average monthly search volume for the title
        point_value = trend_data[title].mean()
    except:
        point_value = 10 # If there is an error, return a point value of 10
    return point_value

# Example usage:
point_value = get_google_trends_score("French Revolution")
print(point_value)


65.0536398467433


In [2]:
from sumy.parsers.plaintext import PlaintextParser
# LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# LsaSummarizer
from sumy.summarizers.lsa import LsaSummarizer
# LuhnSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
# TextRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import textwrap
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

# import minmax scaler eventually for scaling the point values between 1 and 10



def generate_card(title, definition, points):
    # determine the font size based on the length of the definition
    
    # definition needs to be extracted from ['definition'] to be used in the function
    if isinstance(definition, list):
        definition = definition[0]
    elif isinstance(definition, str):
        # replace '[', and ']' with ''
        definition = definition.replace('[', '')
        definition = definition.replace(']', '')
        definition = definition.replace("'", '')
        definition = definition.replace('"', '')
    
    body_font_size = int(len(definition) / 20)
    title_font_size = 10

    # create the image and draw objects
    image = Image.new('RGB', (400, 300), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    # select a font and draw the title in a rectangle
    # font = ImageFont.truetype('./fonts/arial.ttf', font_size)
    # add fontsize for title
    font = ImageFont.truetype('./fonts/NewYork.ttf', title_font_size)
    # add fontsize for body
    font = ImageFont.truetype('./fonts/NewYork.ttf', body_font_size)
    draw.rectangle([(10, 10), (390, 50)], fill='lightgrey')
    draw.text((20, 20), title, fill=(0, 0, 0))

    # draw the definition in a rectangle
    wrapped_definition = textwrap.wrap(definition, width=50)
    wrapped_definition_str = "\n".join(wrapped_definition)
    draw.text((20, 70), wrapped_definition_str, fill=(0, 0, 0))

    draw.rectangle([(10, 60), (390, 250)])
    draw.text((20, 70), wrapped_definition, fill=(0, 0, 0))


    # draw a circle around the point value
    draw.ellipse([(360, 270), (390, 300)], fill='lightgrey')
    draw.text((365, 275), str(points), fill=(0, 0, 0))

    # save the image
    image.save('./card_images/{}.png'.format(len(os.listdir('./card_images/'))))

# todo
def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

import textwrap


import json
# open the cards file
with open("ppn_deck_cleaned.json", "r") as read_file:
    card_deck = json.load(read_file)

# create a new card deck (cards with point values from Google Trends)
new_card_deck = []

# loop through the cards in the card deck
for card in tqdm(card_deck):
    # get the title and definition
    title = card["title"]
    definition = summarize_text(card["summary"],2)
    # check if the definition has more than 2 sentences
    if len(definition.split(".")) > 2:
        # if so, use the first 2 sentences
        definition = ".".join(definition.split(".")[:2])
    
    # get the Google Trends score for the title
    points = get_google_trends_score(str(title))
    # round the points to the nearest integer
    points = int(round(points))
    # create a new card with the title, definition, and point value
    new_card = {
        "title": title,
        "definition": definition,
        "points": points
    }
    # add the new card to the new card deck
    card_generated = generate_card(title, definition, str(points)) # generate card image
    print(f'title: {title}, definition: {definition}, points: {points}')
    #create_card(title, definition, str(points), 400, 150) # generate card image




  0%|          | 1/931 [00:00<07:27,  2.08it/s]

title: Climate change and cities, definition: Moreover, because most cities have been built on rivers or coastal areas, cities are frequently vulnerable to the subsequent effects of sea level rise, which cause coastal flooding and erosion, and those effects are deeply connected with other urban environmental problems, like subsidence and aquifer depletion.\nA report by the C40 Cities Climate Leadership Group described consumption based emissions as having significantly more impact than production-based emissions within cities, points: 26


  0%|          | 2/931 [00:01<08:05,  1.91it/s]

title: Ski-U-Mah (slogan), definition: ['Ski-U-Mah (slogan)', 'Ski-U-Mah SKY-yoo-MAH is a slogan used at the University of Minnesota since 1884, when the newly emerging football team was coached by Thomas Peebles, a philosophy professor and former Princeton University faculty member.\nDuring the teams scrimmages, Peebles would often yell Sis-Boom-Ah, Princeton!
They used the cheer in their practices and games, and the campus paper the Ariel endorsed the cheer soon after, points: 10


  0%|          | 3/931 [00:02<12:06,  1.28it/s]

...

title: Life Cube Project, definition: ['Life Cube Project', 'The Life Cube Project is a community interactive art installation based upon the creators idea that if you write something down, it is far more likely to happen.
The Life Cube installation encourages members of the community and the general public to decorate it with inspirational writing, paintings, drawings, murals, and tapestries for all its visitors to see, points: 8


  7%|▋         | 63/931 [00:46<10:09,  1.43it/s]

title: Schlager music, definition: ['Schlager music', 'Schlager music German: la, hits is a style of European popular music that is generally a catchy instrumental accompaniment to vocal pieces of pop music with simple, happy-go-lucky, and often sentimental lyrics.\n\nTypical Schlager tracks are either sweet, sentimental ballads with a simple, catchy melody or light pop tunes, points: 35


  7%|▋         | 64/931 [00:46<08:36,  1.68it/s]

title: Bombay Progressive Artists' Group, definition: ["Bombay Progressive Artists' Group", 'The Progressive Artists Group PAG, was a group of modern artists, mainly based in Bombay, from its formation in 1947.
Though it lacked any particular style, there might be said to have been a move towards a synthesis of influences from Indian art history together with styles prevalent in Europe and North America during the first half of the 20th century, including Post-Impressionism, Cubism and Expressionism, points: 10


In [ ]:
from nltk.metrics.distance import edit_distance
import json
import nltk
from nltk.corpus import wordnet
import random
import nltk
from tqdm import tqdm
from nltk.corpus import wordnet
stopwords = nltk.corpus.stopwords.words('english')
def find_synonyms(word):
    synonyms = []
    lemmatizer = nltk.WordNetLemmatizer()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name()
            synonyms.append(synonym)
    return set(synonyms)

def replace_similar_words(title, definition, threshold=3):
    similar_words = find_similar_words(title, definition, threshold)
    
    # replace each similar word with a random synonym
    for word in similar_words:
        synonyms = find_synonyms(word)
        synonym = random.choice(list(synonyms))  # choose a random synonym from the set
        print(f"Replacing {word} with {synonym}")
        definition = definition.replace(word, synonym)
        
    return definition


# Example usage
print(find_synonyms("abolitionism"))
# Output: {'abolition', 'abolitionary', 'abolitionism', 'manumission', 'emancipation'}

# open the card deck file
with open("ppn_deck.json", "r") as read_file:
    card_deck = json.load(read_file)
def find_similar_words(title, definition, threshold=3):
    # split the definition into a list of words
    if isinstance(definition, list):
        words = definition
    else:
        words = definition.split()
    
    # use a list comprehension to create a list of words with a Levenshtein distance less than the threshold
    similar_words = [word for word in words if edit_distance(title, word) <= threshold]
    
    return similar_words

def replace_similar_words(title, definition, threshold=3):
    similar_words = find_similar_words(title, definition, threshold)
    if isinstance(definition, list):
        definition = " ".join(definition)
    # replace each similar word with a synonym
    for word in similar_words:
        if len(word) < 3 or word in stopwords:
            continue
        try:
            synonyms = find_synonyms(word) # find a synonym for the word -> set
            # extract the synonym from the set
            synonyms = list(synonyms)
            synonym = synonyms[0]
            #print(f"Replacing {word} with {synonym}")
            definition = definition.replace(word, str(synonym))
        except Exception as e:
            #print(e)
            #print("No synonyms found for word: ", word)
            pass
    return definition

# iterate through each card and replace the similar words in the summary_short with synonyms
for card in tqdm(card_deck):
    title = card['title']
    definition = card['summary']
    definition = replace_similar_words(title, definition)
    
    # update the definition in the card dictionary
    card['summary_short'] = definition


{'abolitionism'}


100%|██████████| 931/931 [00:12<00:00, 73.19it/s] 


In [ ]:
with open("ppn_deck_cleaned.json", "w") as write_file:
    json.dump(card_deck, write_file, indent=4)

# create a csv file with the card titles and definitions and point values
import csv
with open('ppn_deck_cleaned.csv', 'w', newline='') as csvfile:
    fieldnames = ['title', 'definition', 'points']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for card in card_deck:
        writer.writerow({'title': card['title'], 'definition': card['summary_short'], 'points': card['points']})



KeyError: 'points'